## **Geoff's ok Kernel**

Just a big mess at the moment

This page does some bad image manipulation and turns the images into a batch list for ml processing.

It crops the black and wide edges, resizes to 512 and poorly gamma corrects.

Models still need to be saved and turned and used to process the test images.

In [ ]:
#%load_ext tensorboard.notebook
#%tensorboard --logdir logs

#import a bunch of rubbish
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow
import tensorflow as tf
import csv as csv
from IPython.display import display
import cv2 
from matplotlib import pyplot as plt
import operator
from keras.models import Model
from keras.layers import *
from keras.activations import relu
from keras.preprocessing.image import ImageDataGenerator
from keras import backend
import os
from random import shuffle 
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from sklearn.metrics import confusion_matrix   
import math
    
    
height = 512
width = 512
circle_img = np.zeros((height,width), np.uint8)
cv2.circle(circle_img,(int(width/2),int(height/2)),255,1,thickness=-1)    
    
start = time.time()

#LOSS AND METRICS FOR LEARNING ALGORITHM
def gacc(y_true, y_pred): #general accuracy
    return 1-(backend.mean(backend.abs(y_pred - y_true), axis=-1))

def right_acc(y_true, y_pred): #general accuracy
    return 1-(backend.mean(backend.minimum(backend.abs((backend.round(y_pred*4) - y_true*4)*4),1), axis=-1))

def mean_absolute_error2(y_true, y_pred): #custom loss algorithm, actually a modded MAE
    return backend.mean(backend.maximum((backend.abs(y_pred - y_true))-.124,0), axis=-1)

def std_error(y_true, y_pred): #std of the error
    return backend.std((backend.abs(y_pred - y_true)))

def p_meanX(y_true, y_pred): #mean of all predictions - should ideally be .5
    return backend.mean(y_pred, axis=-1)

def p_stdX(y_true, y_pred): #std of all predictions - should be ~0.32 to 0.35
    return backend.std(y_pred)

# Aaaaaaaaaaaaaaaaa

end = time.time()
print(end - start)

In [ ]:
## Callable functions -- just shrink and hide this junk

def adjust_gamma(image, gamma=1.0): #old version of gamma correction
    i = 0
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

def adjust_gamma2(img, gamma=1.0): #new colour balancer, normalises colour balance
    new = [20,80,120]
    newimge = np.zeros(512*512*3)
    newimge = np.reshape(newimge,(512,512,3))
    newimge[:,:,0] = img[:,:,0]*(min(2,(new[0] / np.mean(img[128:384,:,0]))))
    newimge[:,:,1] = img[:,:,1]*(min(2,(new[1] / np.mean(img[128:384,:,1]))))
    newimge[:,:,2] = img[:,:,2]*(min(2,(new[2] / np.mean(img[128:384,:,2]))))
    newimge = np.clip(newimge,0,254)
    img = newimge.astype(np.uint8)
    return img

def autocrop(image, threshold=10): #is meant to get rid of the back borders (works most times)
    if len(image.shape) == 3:
        flatImage = np.max(image, 2)
    else:
        flatImage = image
    assert len(flatImage.shape) == 2

    rows = np.where(np.max(flatImage, 0) > threshold)[0]
    if rows.size:
        cols = np.where(np.max(flatImage, 1) > threshold)[0]
        image = image[cols[0]: cols[-1] + 1, rows[0]: rows[-1] + 1]
    else:
        image = image[:1, :1]
    return image

def im_resize(img):
    img = autocrop(img)
    height,width, x = img.shape
    margin = 0
    if width <15 or height < 15:
        img = zeros(512*512*3).reshape(512,512,3)
        height,width, x = img.shape
    if(width>=height):
        margin = int(np.floor((width-height)/2))
        #print(height, width, margin)
        crop_img = img[0:height,(width-height-margin):(height+margin)]
    else:
        margin = int(np.floor((height-width)/2))
        #print(height, width, margin)
        crop_img = img[(height-width-margin):(width+margin),0:width]
    img = cv2.resize(crop_img,(512,512),interpolation=cv2.INTER_AREA)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    return img

#im_avg returns a tuple of the inner area of the cropped 512x512 image
def im_avg(img):
    return cv2.mean(img[100:412,100:412])

end = time.time()
print(end - start)

## ML Time!
### the Model

Labels and training images are loaded and done



In [ ]:
### Need to make a dataframe to do dataframe access of images and classications

# probably should have done this in the image gen script

dirnam="/kaggle/input/finalimagese/data"
dirnum="/0/"
dirname=dirnam+dirnum
for dirname, _, filenames in os.walk(dirname): pass
dls0 = pd.DataFrame()
dls0['name'] = filenames
dls0['name'] = dirname + dls0['name'].astype(str)
dls0['diagnosis'] = 0

dirnum="/1/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dls1 = pd.DataFrame()
dls1['name'] = filenames
dls1['name'] = dirnam+dirnum + dls1['name'].astype(str)
dls1['diagnosis'] = .25

dirnum="/2/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dls2 = pd.DataFrame()
dls2['name'] = filenames
dls2['name'] = dirnam+dirnum + dls2['name'].astype(str)
dls2['diagnosis'] = .5

dirnum="/3/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dls3 = pd.DataFrame()
dls3['name'] = filenames
dls3['name'] = dirnam+dirnum + dls3['name'].astype(str)
dls3['diagnosis'] = .75

dirnum="/4/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dls4 = pd.DataFrame()
dls4['name'] = filenames
dls4['name'] = dirnam+dirnum + dls4['name'].astype(str)
dls4['diagnosis'] = 1

dirnam="/kaggle/input/finalimagese/data"
dirnum="/0/"
dirname=dirnam+dirnum
for dirname, _, filenames in os.walk(dirname): pass
dlsA0 = pd.DataFrame()
dlsA0['name'] = filenames
dlsA0['name'] = dirnam+dirnum + dlsA0['name'].astype(str)
dlsA0['diagnosis'] = 0

dirnum="/1/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dlsA1 = pd.DataFrame()
dlsA1['name'] = filenames
dlsA1['name'] = dirnam+dirnum + dlsA1['name'].astype(str)
dlsA1['diagnosis'] = .25

dirnum="/2/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dlsA2 = pd.DataFrame()
dlsA2['name'] = filenames
dlsA2['name'] = dirnam+dirnum + dlsA2['name'].astype(str)
dlsA2['diagnosis'] = .5

dirnum="/3/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dlsA3 = pd.DataFrame()
dlsA3['name'] = filenames
dlsA3['name'] = dirnam+dirnum + dlsA3['name'].astype(str)
dlsA3['diagnosis'] = .75

dirnum="/4/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dlsA4 = pd.DataFrame()
dlsA4['name'] = filenames
dlsA4['name'] = dirnam+dirnum + dlsA4['name'].astype(str)
dlsA4['diagnosis'] = 1


dirnam="/kaggle/input/finalimagese/data/data"
dirnum="/0/"
dirname=dirnam+dirnum
for dirname, _, filenames in os.walk(dirname): pass
dlsB0 = pd.DataFrame()
dlsB0['name'] = filenames
dlsB0['name'] = dirnam+dirnum + dlsB0['name'].astype(str)
dlsB0['diagnosis'] = 0

dirnum="/1/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dlsB1 = pd.DataFrame()
dlsB1['name'] = filenames
dlsB1['name'] = dirnam+dirnum + dlsB1['name'].astype(str)
dlsB1['diagnosis'] = .25

dirnum="/2/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dlsB2 = pd.DataFrame()
dlsB2['name'] = filenames
dlsB2['name'] = dirnam+dirnum + dlsB2['name'].astype(str)
dlsB2['diagnosis'] = .5

dirnum="/3/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dlsB3 = pd.DataFrame()
dlsB3['name'] = filenames
dlsB3['name'] = dirnam+dirnum + dlsB3['name'].astype(str)
dlsB3['diagnosis'] = .75

dirnum="/4/"
for dirname, _, filenames in os.walk(dirnam+dirnum): pass
dlsB4 = pd.DataFrame()
dlsB4['name'] = filenames
dlsB4['name'] = dirnam+dirnum + dlsB4['name'].astype(str)
dlsB4['diagnosis'] = 1

dls = pd.DataFrame().append([dlsA0, dlsA1, dlsA2, dlsA3, dlsA4, dlsB0, dlsB1, dlsB2, dlsB3, dlsB4],ignore_index = True)
dlsV = pd.DataFrame().append([dls0, dls1, dls2, dls3, dls4],ignore_index = True)
dls['name'] = dls['name'].astype(str)

dls.reset_index(drop=True)
print(dls)
dlsV = dlsV.sample(len(dlsV))
dls = dls.sample(len(dls))

end = time.time()
print(end - start)

In [ ]:
def inception(x,depth):
    x1 = Conv2D(depth, (1, 1), activation='relu', padding='same')(x)
    x2 = Conv2D(depth, (1, 1), activation='relu', padding='same')(x)
    x3 = Conv2D(depth, (1, 1), activation='relu', padding='same')(x)
    x4 = Conv2D(depth, (1, 1), activation='relu', padding='same')(x)
    x2 = Conv2D(depth, (1, 3), activation='relu', padding='same')(x2)
    x2 = Conv2D(depth, (3, 1), activation='relu', padding='same')(x2)
    x3 = Conv2D(depth, (1, 5), activation='relu', padding='same')(x3)
    x3 = Conv2D(depth, (5, 1), activation='relu', padding='same')(x3)
    x4 = Conv2D(depth, (1, 7), activation='relu', padding='same')(x4)
    x4 = Conv2D(depth, (7, 1), activation='relu', padding='same')(x4)
    x3a = AveragePooling2D(pool_size=(3, 3), strides=1, padding='same')(x)
    x3b = Conv2D(depth, (1, 1), activation='relu', padding='same')(x3a)
    d = concatenate([x1,x2,x3,x4,x3b], axis=-1)
    return d

def reductionF(x,depth):
    x1 = Conv2D(depth, (1, 1), activation='relu', padding='same')(x)
    x1 = Conv2D(depth, (3, 3), strides=2, padding='same')(x1)
    x2 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
    x2 = Conv2D(depth, (1, 1),  padding='same')(x2)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    d = concatenate([x1,x2], axis=-1)
    return d

def reductionI(x):
    x1 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x)
    x2 = AveragePooling2D(pool_size=(2, 2), strides=2, padding='same')(x)
    d = concatenate([x1,x2], axis=-1)
    return d

def reductionG(x,depth):
    x1 = Conv2D(depth, (1, 1), activation='relu', padding='same')(x)
    x1 = Conv2D(64, (3, 3), strides=2, padding='same')(x1)
    x2 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
    x2 = Conv2D(depth, (1, 1), padding='same')(x2)
    x3 = Conv2D(depth, (1, 1), activation='relu', padding='same')(x)
    x3 = Conv2D(depth, (1, 7), activation='relu', padding='same')(x3)
    x3 = Conv2D(depth, (7, 1), activation='relu', padding='same')(x3)
    x3 = Conv2D(64, (3, 3), strides=2, padding='same')(x3)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    x3 = BatchNormalization()(x3)
    x3 = Activation('relu')(x3)
    d = concatenate([x1,x2,x3], axis=-1)
    return d


a = Input(shape=(512,512,3))
y = BatchNormalization()(a)
y1= reductionI(y)
x = reductionF(y, 32)
y= reductionI(y1)
x = reductionF(x, 64)
x1 = inception(x,64)
x1 = Conv2D(64, (1, 1))(x1)
x = concatenate([x1,x,y], axis=-1,name='128x128xinception')
x = GaussianNoise(0.2)(x)
y= reductionI(y)
x = reductionG(x, 64)
x1 = inception(x,64)
#x1 = inception(x1,64)
x1 = Conv2D(64, (1, 1))(x1)
x = concatenate([x1,x,y], axis=-1,name='64x64xinception2')
y= reductionI(y)
x = reductionG(x, 64)
x1 = inception(x,64)
#x1 = inception(x1,64)
x1 = Conv2D(64, (1, 1))(x1)
x = concatenate([x1,x,y], axis=-1,name='32x32xinception2')
x = GaussianNoise(0.5)(x)
y= reductionI(y)
x = reductionG(x, 64)
x1 = inception(x,64)
#x1 = inception(x1,64)
x1 = Conv2D(64, (1, 1))(x1)
x = concatenate([x1,x,y], axis=-1,name='16x16xinception2')
x = reductionG(x, 64)
x1 = inception(x,64)
#x1 = inception(x1,64)
x1 = Conv2D(64, (1, 1))(x1)
x = concatenate([x1,x], axis=-1,name='8x8xinception2')
x = GaussianNoise(0.5)(x)
x = reductionG(x, 64)
x = GaussianNoise(0.3)(x)
x = Conv2D(32, (1, 1),name='4x4xconvolution')(x)
x = AveragePooling2D(pool_size=(4, 4), strides=4, padding='valid')(x)
x = Conv2D(1, (1, 1),name='1x1xconvolution')(x)
x = Flatten()(x)
b = Dense(1,activation='sigmoid')(x)
model = Model(inputs=a, outputs=b)

model.summary()

model_rebuilt =1

end = time.time()
print(end - start)

In [ ]:
datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True
)
the_flow = datagen.flow_from_dataframe(dls,x_col='name',y_col='diagnosis',shuffle=True, class_mode='raw',target_size=(512, 512), batch_size=32)
the_valid = datagen.flow_from_dataframe(dlsV,x_col='name',y_col='diagnosis',shuffle=True, class_mode='raw',target_size=(512, 512), batch_size=32)

end = time.time()
print(end - start)

### Model Generation Time!

In [ ]:
#Running a model


filepath="TheBestModel.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_right_acc', verbose=0, save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint("TheBestLoss.hdf5", monitor='val_loss', verbose=0, save_best_only=True, mode='min')
checkpoint3 = ModelCheckpoint("TheBestStd.hdf5", monitor='val_std_error', verbose=0, save_best_only=True, mode='min')
checkpoint4 = ModelCheckpoint("TheBestTVAL.hdf5", monitor='val_std_error', verbose=0, save_best_only=True, mode='min')
checkpoint5 = ModelCheckpoint("TheBestTLOSS.hdf5", monitor='val_std_error', verbose=0, save_best_only=True, mode='min')
checkpoint6 = ModelCheckpoint("TheBestTSTD.hdf5", monitor='val_std_error', verbose=0, save_best_only=True, mode='min')
tensorboard_callback = tf.keras.callbacks.TensorBoard("logs")

callbacks_list = [checkpoint,checkpoint2,checkpoint3,checkpoint4,checkpoint5,checkpoint6,tensorboard_callback]

sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=[right_acc, std_error,p_meanX,p_stdX ])
#datagen.fit(train_images)

model_rebuilt =0

#model.fit(x=train_images, y=train_labels, epochs=50, verbose =0,validation_split=0.1, shuffle=True) 

end = time.time()
print(end - start)
firstrun = 1

In [ ]:
batch_size = 32
epochs = 100
verboseX = 2
steps_per_e = 200
validation_stepsX = 20

if firstrun == 1:
    !cp /kaggle/input/goodkernel/TheBestLoss.hdf5 /kaggle/working/TheBestModel.hdf5
    model.load_weights(filepath)
    
firstrun = 0
hist = model.fit_generator(the_flow, callbacks=callbacks_list, use_multiprocessing=False, shuffle=True, workers=1,steps_per_epoch=steps_per_e,
               epochs=1, verbose =verboseX, validation_data=the_valid, validation_steps=validation_stepsX )
hist = model.fit_generator(the_flow, callbacks=callbacks_list, use_multiprocessing=False, shuffle=True, workers=1,steps_per_epoch=steps_per_e,
               epochs=epochs, verbose =verboseX, validation_data=the_valid, validation_steps=validation_stepsX )

model.load_weights(filepath)

end = time.time()
print(end - start)


In [ ]:
plt.plot(hist.history['right_acc'])
plt.plot(hist.history['val_right_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

end = time.time()
print(end - start)

## Sanity Check
Checks the model against all the training data - as we only used a subsample of 1000 images, that leaves 2662 images as well as the originals
good is good, bad is bad and the array is which images were not detected properly

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/aptos2019-blindness-detection/train_images'):
    pass
dirname="/kaggle/input/aptos2019-blindness-detection/train_images"
    
train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv', header=0)
train['diagnosis_2'] = 0
train['diagnosis_3'] = 0
bad = [0,0,0,0,0]
total_type = [0,0,0,0,0]

w = np.zeros((5,5)); w
for i in range(len(w)):
    for j in range(len(w)):
        w[i][j] = float(((i-j)**2)/16) #as per formula, for this competition, N=5

actuals = []
pred_list = []

count = 0
accurat = 0
inaccurat = 0
#nimg = {}
for filename in filenames:
    if filename[(len(filename)-4):] == ".png":
        img = cv2.imread(os.path.join(dirname, filename))
        new_img = im_resize(img)
        thisav = im_avg(new_img)
        dgamma = thisav[0]+thisav[1]+thisav[2]
        #new_img = adjust_gamma2(new_img,(((400) / dgamma)))
        #new_img = np.divide(new_img,255.0)
        count = count+1
        k = np.reshape(new_img,(1,512,512,3))
        
        real_pred = min(4,max(0,model.predict(k)[0][0]*4))
        pred = round(real_pred)  #np.round(real_pred)
        real = train[train.id_code==filename[:-4]].diagnosis.item()
        train.loc[train.id_code==filename[:-4],'diagnosis_2']=int(pred)
        train.loc[train.id_code==filename[:-4],'diagnosis_3']=real_pred
        #print(real, " ", pred, " ",real_pred)
        actuals.append(real)
        pred_list.append(int(pred))
        total_type[real] +=1
        if int(pred) == real:
            accurat +=1
        else:
            inaccurat +=1
            bad[real] += 1
        if count % 100 == 0: 
            O = confusion_matrix(actuals, pred_list); print(O)
            N=5
            act_hist=np.zeros([N])
            for item in actuals: 
                act_hist[item]+=1

            pred_hist=np.zeros([N])
            for item in pred_list: 
                pred_hist[item]+=1
            E = np.outer(act_hist, pred_hist); E
            E = E/E.sum(); E.sum()
            O = O/O.sum(); O.sum()                    
            num=0
            den=0
            for i in range(len(w)):
                for j in range(len(w)):
                    num+=w[i][j]*O[i][j]
                    den+=w[i][j]*E[i][j]

            weighted_kappa = (1 - (num/den)); print(weighted_kappa)
            
            print(count, " - Good: ",accurat, " n Bad: ",inaccurat, " - Bad: ",bad, " Total = ",total_type, " percent right = ",[1 - ((b) / (m)) for b,m in zip(bad, total_type)])
 
train

end = time.time()
print(end - start)

## Final Output Processing
This runs over the test sets and makes a final result submission.csv

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/aptos2019-blindness-detection/test_images'):
    pass
dirname="/kaggle/input/aptos2019-blindness-detection/test_images"
    
test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv', header=0)
test['diagnosis'] = 0

test_images = []
test_labels = []
count = 0
#nimg = {}
for filename in filenames:
    if filename[(len(filename)-4):] == ".png":
        img = cv2.imread(os.path.join(dirname, filename))
        new_img = im_resize(img)
        thisav = im_avg(new_img)
        dgamma = thisav[0]+thisav[1]+thisav[2]
        #new_img = adjust_gamma2(new_img,(((400) / dgamma)))
        #new_img = np.divide(new_img,255.0)
        count = count+1
        #nimg[filename[:-4]] = new_img
        k = np.reshape(new_img,(1,512,512,3))
       # print(np.shape(k), np.shape(test_images))
        #if len(test_images) ==0:
        #    test_images = np.copy(k)
        #else:
        #    test_images=np.vstack([test_images,k])
        #test_labels.append(test[test.id_code==filename[:-4]].diagnosis.item())
        real_pred = min(4,max(0,model.predict(k)[0][0]*4))
        pred = round(real_pred)  #np.round(real_pred)
        test.loc[test.id_code==filename[:-4],'diagnosis']=int(pred)
       # print(pred, int(pred))
        
        if count % 50 == 0: 
            print(count)
     #   if count % 1000 == 0: 
            #break
#test_labels = np.asarray(test_labels)    
test.to_csv('submission.csv',index=False)
test
#pred = np.round(abs(model.predict(test_images))*4)
#print(pred)

end = time.time()
print(end - start)

In [ ]:
a = np.round([8,3.56,4,5,6])
print(a.sort())
print(a)